In [16]:
from kagglehub import dataset_download
import pandas as pd
import seaborn as sns

In [5]:
path = dataset_download("yufengdev/bbc-fulltext-and-category")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\nnn866470\.cache\kagglehub\datasets\yufengdev\bbc-fulltext-and-category\versions\2


In [7]:
df = pd.read_csv(path+ "/bbc-text.csv")

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  2225 non-null   object
 1   text      2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB


In [11]:
df.describe()

,category,text
count,2225,2225
unique,5,2126
top,sport,europe backs digital tv lifestyle how people r...
freq,511,2


In [12]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [15]:
df['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

Check for class weightage